In [ ]:
# Download example data

import obspy
from obspy.clients.fdsn.client import Client
import os
os.mkdir('./example_data/')
# one-station two-day Ridgecrest sequence data
# you should be able to detect 4028 events from this example using DiTingPicker.
starttime = obspy.UTCDateTime('2019-07-06T00:00:00')
endtime = obspy.UTCDateTime('2019-07-08T00:00:00')
network = 'CI'
station = 'CLC'

client = Client('IRIS')

inv_info = client.get_stations(network=network, station=station, channel='HH*', location='*', level='response')
inv_info.write('./example_data/inv.xml',format='STATIONXML')
two_day_data = client.get_waveforms(network=network, station=station, channel='HH*', location='*', starttime=starttime, endtime=endtime) 
two_day_data.write('./example_data/waveform.mseed')

In [1]:
# set gpu
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '7'
from example_codes_for_detecting_earthquakes import *

In [2]:
st = obspy.read('./example_data/waveform.mseed')
sta_dict = dict()
st.merge(fill_value='interpolate')
st.trim(starttime=st[0].stats.starttime, endtime=st[0].stats.endtime, pad=True, fill_value=0)

for trace in st.traces:
    net = trace.stats.network
    sta = trace.stats.station
    net_sta_key = net + '.' + sta
    if net_sta_key in sta_dict.keys():
        pass
    else:
        sta_dict[net_sta_key] = list()
        
sta_dict_key_list = list(sta_dict.keys())
input_dict = dict()
input_dict['st'] = st
input_dict['sta_dict'] = sta_dict
input_dict['fname'] = 'example'
input_dict['plot_folder'] = './plots/'
input_dict['waveform_folder'] = './results/waveforms/'
input_dict['csv_folder'] = './results/csvs/'
input_dict['GPU_ID'] = '7'

picker_model_path = './DiTingPicker_2022_LuDing_sequence.hdf5'
input_dict['picker_model_path'] = picker_model_path
picker_model = tf.keras.models.load_model(picker_model_path, compile=False)

motion_model_path = './DiTingMotion_2022_LuDing_sequence.hdf5'
input_dict['motion_model_path'] = motion_model_path
motion_model = tf.keras.models.load_model(motion_model_path, compile=False)

input_dict['max_psdiff'] = 50
input_dict['min_psdiff'] = 0.5
input_dict['sta_dict_key_list'] = sta_dict_key_list
input_dict['show_mode'] = False

2022-12-28 20:14:09.874982: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-28 20:14:10.604661: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22296 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:d6:00.0, compute capability: 8.6


In [3]:
sta_dict = process_sta_list_passing_model(input_dict, picker_model, motion_model, None)
fp = open('./results/example.json', "w")
json.dump(sta_dict, fp)
fp.close()
print('Done')

# Please note that picked phases are ploted one event per figure.

2022-12-28 20:14:17.204292: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-12-28 20:14:20.759802: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8401
2022-12-28 20:14:23.723041: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


Done
